## 1. CNN简介  
卷积神经网络（Convolutional Neural Network，CNN）是一种前馈神经网络，它的神经元可以响应一部分覆盖范围内的周围单元，对于图像有出色的表现。相比较于全连接网络，还包括卷积层（convolutional layer）和池化层（pooling layer）。

### 1.1 卷积运算 
以**Conv2D**为例   
卷积神经网络接收形状为`(image_height, image_width, image_channels)`的输入张量（不包括批量维度）。对于包含两个空间轴（高度和宽度）和一个深度轴（通道轴）的3D张量，其卷积也称之为**特征图(feature map)**。输出的特征图也是仍然是一个3D张量，具有高度和宽度，其深度可以是任意取值（设定的参数），此时深度不再像RGB图像那样表示特定的颜色，而是代表**过滤器**。可以理解为过滤器对输入的数据的某一方面进行编码。   

> Note: 特征图表示某个过滤器（卷积核）在输入中不同位置的响应，深度轴上每个维度都是一个特征

- **卷积工作原理**    
在3D输入特征图上滑动，按照卷积核的大小进行滑动，在每个可能的位置停止并提取周围特征的3D图块（window_height, window_width, input_depth）。然后每个3D图块与学到的同一个权重矩阵（卷积核）做张量积，转换为(output_depth, )的1D向量。然后对这些向量进行空间重组，使其转换为形状(height, width, output_depth)的3D输出特征图。过程参考下图， 
![](../../../pics/卷积工作原理.png)

#### 1.1.1 卷积中关键参数   
- 卷积核的大小    
从输入中提取的图块的大小，通常是$3 \times 3$或$5 \times 5$

- 过滤器的数量    
输出特征图的深度

- 填充(padding)   
假设有$6 \times 6$的特征图（36个方块），如果以$3 \times 3$的卷积核进行滑动，最终输出的特征图的尺寸是$4 \times 4$，也就是说比输入的尺寸小了一些。如果希望输入和输出的特征图空间维度相同，那么可以使用**填充**。   
填充的具体做法是在输入特征图的每一边添加适当数目的行和列，使得每个输入方块都能作为卷积窗口的中心。   
所以padding参数有两个取值，valid和same，valid表示不使用填充，same表示填充后输出和输入的维度相同。

- 步幅(stride)   
卷积时，两个连续窗口的距离是卷积的一个参数，步幅，默认值为1，可以使用步进卷积(strided convolution)，即步幅大于1的卷积。    
步幅为2意味着特征图的宽度和高度都被做了2倍下采样。虽然步进卷积对于一些模型可能有用，但在实践中很少使用。  
为了对特征图进行下采样，通常不使用步幅，而是使用池化(max-pooling)。

卷积的可视化图，
<table style="width:100%; table-layout:fixed; text-align: center;">
  <tr>
    <td>No padding, no strides</td>
    <td>Arbitrary padding, no strides</td>
    <td>Half padding, no strides</td>
    <td>Full padding, no strides</td>
  </tr>
  <tr>
    <td><img width="200px" src="../../../pics/no_padding_no_strides.gif"></td>
    <td><img width="200px" src="../../../pics/arbitrary_padding_no_strides.gif"></td>
    <td><img width="200px" src="../../../pics/same_padding_no_strides.gif"></td>
    <td><img width="200px" src="../../../pics/full_padding_no_strides.gif"></td>
  </tr>
  <tr>
    <td>No padding, strides</td>
    <td>Padding, strides</td>
    <td>Padding, strides (odd)</td>
  </tr>
  <tr>
    <td><img width="200px" src="../../../pics/no_padding_strides.gif"></td>
    <td><img width="200px" src="../../../pics/padding_strides.gif"></td>
    <td><img width="200px" src="../../../pics/padding_strides_odd.gif"></td>
  </tr>
</table>

#### 1.1.2 卷积的性质  
- 模式的空间层次结构       
卷积神经网络可以学到模式的**空间层次结构(spatial hierarchies of patterns)**。往往开始的卷积层学习较小的局部模式（比如眼睛，嘴巴等），接下来的卷积层将学习组成上一层的特征模式，依次类推。这使得卷积神经网络可以有效地学习越来越复杂、抽象的视觉概念。（视觉世界从根本上具有空间层次结构）   

- 平移不变性    
全连接层从特征空间中学到的是全局模式，而**卷积层学到的局部模式**。如果学习到这个局部模式后，它可以在任何地方识别这个模式(视觉世界从根本上具有平移不变性)。这可以使得卷积神经网络可以高效地利用数据，使用较少的训练样本可以学到具有泛化能力的数据表示。

### 1.2 最大池化运算  
最大池化是从输入特征图中提取窗口，并输出每个通道的最大值。它的概念与卷积类似，不同的是最大池化使用硬编码的max张量运算对局部图块进行变换，而卷积则是使用学到的线性变换（卷积核）。    
从参数上看，卷积和最大池化的不同是最大池化通常使用$2 \times 2$的窗口和步幅2，其目的是将图像下采样2倍，卷积则是使用$3 \times 3$的窗口和步幅1。   

#### 1.2.1 为什么不删除最大池化层，保留较大的特征图？    
1）不利于学习特征的空间层级结构，如果去掉池化层，这样卷积神经网络从输入中提取的原始图像比例随着层数加深在减少，不足以相应任务的学习。    
2）学习参数较多，最后一层参数在全连接后会很多。   

#### 1.2.2 使用最大池化的作用   
1）减少需要处理的元素的个数，加速网络的训练，防止过拟合   
2）通过让连续池化层，可以让观测窗口越来越大（即窗口覆盖原始输入的比例越来越大），从而引入空间过滤器的层级结构  

#### 1.2.3 相较于平均池化，为什么最大池化更好
特征中往往编码了某种模式下或概念在特征图中的不同位置是否存在，而最大值相较于平均值能够给出更多的信息，而平均值方法可能导致错过或淡化特征是否存在的信息。   
合理的下采样策略是首先生成密集的特征图（无步进卷积），然后观察特征在每个小块上的最大激活，而不是查看输入的稀疏窗口（步进卷积）或者对输入图块取平均，这两种方式肯嗯导致错过或者淡化重要特征。

## 2. CNN网络结构   
以**ImageNet**数据集的算法演变来总结CNN算法。大致有LeNet(1998),AlexNet(2012),ZFNet(2013),GoogleNet/Inception(2014),VGGNet(2014),ResNet(2015)。

### 2.1 LeNet(1998)
LeNet是卷积神经网络的祖师爷LeCun在1998年提出，用于解决手写数字识别的视觉任务。自那时起，CNN的最基本的架构就定下来了：卷积层、池化层、全连接层。


#### 2.1.1 LeNet网络架构
经典的LeNet-5结构图如下，
![](../../../pics/LeNet.png)

剖析：  
- 第一层，输入层（input），输入图像是单通道的$32 \times 32$大小的图像，张量形状为(1, 32, 32)  

- 第二层，卷积层（conv1），卷积核尺寸为$5 \times 5$，步幅(stride)为1，卷积核的数量为6，经过卷积核后的图像尺寸大小为$(32-5+1) \times (32-5+1) \rightarrow 28 \times 28$，该层输出的形状为(6, 28, 28)  
> Note: 第二层中，使用6个大小为5*5的卷积核，这部分参数为6×(5×5+1)=156，+1表示一个核有bias，总共有156×28×28=122304个连接，但是只需要学习156个参数，主要是通过权值共享实现

- 第三层，最大化池化层（pool1），池化核的大小为$2 \times 2$，步幅为2，池化操作后，图像尺寸减半，变为$14 \times 14$，该层输出后的形状为(6, 14, 14)，训练参数为6×(2×2+1)，连接数量为6×(2×2+1)×14×14   

- 第四层，卷积层（conv2），类似第二层，卷积核尺寸大小为$5 \times 5$，步幅为1,卷积核数量为16,卷积后的图像大小为（14-5+1），该层输出后的形状为(16, 10, 10)
> 疑问：第四层的输入不再是(6, 14, 14)，而是第二层中所有feature map或者几个feature map的组合

- 第五层，池化层（pool2），类似第三层，池化核尺寸大小为$2 \times 2$，步幅为2,池化操作后，图像尺寸减半，变为$5 \times 5$，该层的输出后的形状为(16, 5, 5)    

- 第六层，卷积层（conv3），卷积核的大小为$5 \times 5$，步幅为1,卷积核数量为120,则输出后的形状为(120, 1, 1)
- 第七层，全连接层，84个神经元（对应7×12的比特图）
- 第八层，输出层，神经元数量为10（目标共有10个类别），然后经过`softmax`，最后输出概率。

#### 2.1.2 代码实现   

```python
def LeNet():
    model = Sequential()
    model.add(Conv2D(6,(5,5),strides=(1,1),input_shape=(32,32,1),padding='valid',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(16,(5,5),strides=(1,1),padding='valid',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(120, (5,5), strides=(1,1), padding='valid',activation='sigmoid')
    model.add(Flatten())
    model.add(Dense(500,activation='sigmoid'))
    model.add(Dense(10,activation='softmax'))
    return model
```
> Note: 上述代码copy网上代码，未实例验证，该工作后期补充  

#### 2.1.3 参考资源   
- [LeNet-5详解](https://cuijiahua.com/blog/2018/01/dl_3.html)

### 2.2 AlexNet(2012)
AlexNet在2012年ImageNet竞赛中以超过第二名10.9个百分点的绝对优势一举夺冠，从此深度学习和卷积神经网络名声鹊起，深度学习的研究如雨后春笋般出现，AlexNet的出现可谓是卷积神经网络的王者归来。

#### 2.2.1 AlexNet网络架构  
![](../../../pics/AlexNet.png)
Alex网络结构整体如上图所示，前五层为卷积层，后面三层为全连接层，最终输出的1000类。 

#### 2.2.2 比较LeNet
相比较于LeNet，AlexNet的卷积神经网络总流程固定，只是网络结构还有一些训练技巧上做了很大改进，
主要的改进点：   
- **更深的网络**   
AlexNet共有5层卷积层和3个全连接层，均比LeNet要多   
- **数据增强**   
对原始图像做了随机裁剪，增加模型的泛化能力，避免过拟合   
- **使用Relu**   
使用Relu代替Sigmoid函数，加快收敛速度   
- **使用dropout**   
在网络中加入了dropout策略，dropout具体参考**算法篇/deep_learning/深度学习基础**
- **Local Responce Normalization**    
局部响应归一层的基本思路是，假如这是网络的一块，比如是 13×13×256， LRN 要做的就是选取一个位置，比如说这样一个位置，从这个位置穿过整个通道，能得到 256 个数字，并进行归一化。进行局部响应归一化的动机是，对于这张 13×13 的图像中的每个位置来说，我们可能并不需要太多的高激活神经元。但是后来，很多研究者发现 LRN 起不到太大作用，因为并不重要，而且我们现在并不用 LRN 来训练网络。

#### 2.2.3 代码实现   
```python
def AlexNet():

    model = Sequential()
    model.add(Conv2D(96,(11,11),strides=(4,4),input_shape=(227,227,3),padding='valid',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    model.add(Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000,activation='softmax'))
    return model
```

## Refercences

[1] [CNN Architectures](https://medium.com/@sidereal/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5)

[2] [CNN网络架构演进：从LeNet到DenseNet](https://www.cnblogs.com/skyfsm/p/8451834.html)